In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from datetime import datetime


In [2]:
file_csv = '../raw_data/merge_dfs.csv'
df_raw_data = pd.read_csv(file_csv)

/var/folders/bn/g48wyj9119lb2ytw0l5thk840000gn/T/ipykernel_56275/900801688.py:2: DtypeWarning: Columns (53,62,71,80,89,98,107,116,122,123) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw_data = pd.read_csv(file_csv)


In [3]:
import pipeline_cleaning


In [4]:
clean_data = pipeline_cleaning.clean_data(df_raw_data)
clean_data

/Users/amandamor/code/Harlequinht/which_horse/notebook/pipeline_cleaning.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['failed_to_finish_reason'] = df['failed_to_finish_reason'].apply(lambda x: 0 if pd.isna(x) else 1)
/Users/amandamor/code/Harlequinht/which_horse/notebook/pipeline_cleaning.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['margin'] = df.apply(lambda row: row['distance'] if pd.isna(row['margin']) and (row['win_or_lose'] == 1 or row['failed_to_finish_reason'] == 1) else row['m

,date,event_number,distance,race_class,race_class_normalised,total_prize_money,track_condition,race_type,track_type,barrier,...,15_mins,10_mins,5_mins,3_mins,2_mins,1_min_,post_time,last_traded_price,rating_oficial,current_age
0,2020-10-05,7,1766,5,Handicap,5445,SOFT,FLAT,TURF,6,...,8.2,8.0,7.8,9.2,9.2,7.4,7.0,1000.0,77.0,7.0
1,2020-10-05,7,1766,5,Handicap,5445,SOFT,FLAT,TURF,8,...,95.0,90.0,120.0,140.0,150.0,180.0,180.0,1000.0,77.0,5.0
2,2020-10-05,6,1766,5,Maiden,5410,SOFT,FLAT,TURF,9,...,60.0,50.0,75.0,80.0,75.0,120.0,110.0,1000.0,NaN,5.0
3,2020-10-05,6,1766,5,Maiden,5410,SOFT,FLAT,TURF,2,...,85.0,90.0,95.0,90.0,85.0,100.0,100.0,1000.0,NaN,4.0
4,2020-10-05,6,1766,5,Maiden,5410,SOFT,FLAT,TURF,8,...,510.0,680.0,800.0,960.0,800.0,980.0,1000.0,1000.0,NaN,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182151,2023-09-28,3,1320,2,Handicap,46920,GOOD TO FIRM,FLAT,TURF,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,6.0
182152,2023-09-28,2,1760,4,Handicap,14296,GOOD TO FIRM,FLAT,TURF,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.0,6.0
182153,2023-09-28,2,1760,4,Handicap,14296,GOOD TO FIRM,FLAT,TURF,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.0,7.0
182154,2023-09-28,3,1320,2,Handicap,46920,GOOD TO FIRM,FLAT,TURF,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.0,7.0


In [5]:
list(clean_data.columns)

['date',
 'event_number',
 'distance',
 'race_class',
 'race_class_normalised',
 'total_prize_money',
 'track_condition',
 'race_type',
 'track_type',
 'barrier',
 'finish_position',
 'win_or_lose',
 'margin',
 'gear',
 'jockey_allowance',
 'handicap_weight',
 'jockey_id',
 'tainer_id',
 'dslr',
 'wfa',
 'weight_adjustment',
 'betfair_starting_price',
 'pre_race_master_rating_int',
 'starting_price',
 'id_lewagon',
 'runners',
 'min_price',
 'max_price',
 '15_mins',
 '10_mins',
 '5_mins',
 '3_mins',
 '2_mins',
 '1_min_',
 'post_time',
 'last_traded_price',
 'rating_oficial',
 'current_age']

In [6]:
clean_data.isna().sum()

date                              0
event_number                      0
distance                          0
race_class                        0
race_class_normalised           285
total_prize_money                 0
track_condition                   0
race_type                         0
track_type                        0
barrier                           0
finish_position                 657
win_or_lose                       0
margin                           91
gear                              0
jockey_allowance                  0
handicap_weight                   1
jockey_id                         0
tainer_id                         0
dslr                          15966
wfa                               0
weight_adjustment                 0
betfair_starting_price         5895
pre_race_master_rating_int    20596
starting_price                    0
id_lewagon                        0
runners                        9207
min_price                      9207
max_price                   

In [ ]:
pipeline_cleaning.transforming_data(clean_data)[0]

In [ ]:
clean_data['date'] = pd.to_datetime(clean_data['date'])
clean_data.drop(columns=['jockey_id', 'tainer_id', 'margin', 'finish_position', 'event_number'], axis=1, inplace=True)
clean_data.dropna(inplace=True) #instead of imputer
df_train = clean_data[(clean_data['date'].dt.year != 2022) & (clean_data['date'].dt.year != 2023)]
df_val = clean_data[clean_data['date'].dt.year == 2022]
df_test = clean_data[clean_data['date'].dt.year == 2023]
df_train.drop(columns=['date'], axis=1, inplace=True)
df_val.drop(columns=['date'], axis=1, inplace=True)
df_test.drop(columns=['date'], axis=1, inplace=True)

categorical_col = ['barrier', 'track_condition', 'race_type', 'track_type',
                    'race_class_normalised', 'race_class']
num_col = ['distance', 'total_prize_money', 'jockey_allowance',
            'handicap_weight', 'dslr', 'official rating', 'wfa',
            'weight_adjustment', 'betfair_starting_price',
            'pre_race_master_rating_int', 'starting_price', 'current_age']


In [ ]:
[col for col in df_train.columns if col not in (num_col+categorical_col)]

In [ ]:
df_test

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

In [ ]:
categorical_preprocessor = Pipeline([
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
])
numerical_preprocessor = Pipeline([
    ('scaler', StandardScaler())
])
pipeline = ColumnTransformer([
    ('categorical', categorical_preprocessor, categorical_col),
    ('numerical', numerical_preprocessor, num_col)
], remainder="passthrough", sparse_threshold=0)
pipeline

In [ ]:
pipeline.fit(df_train)
df_train_transformed = pipeline.transform(df_train)
df_val_transformed = pipeline.transform(df_val)
df_test_transformed = pipeline.transform(df_test)

In [ ]:
pipeline.transform(df_train)

In [ ]:
# pipeline.get_feature_names_out()


In [ ]:
categorical_feature_names = pipeline.named_transformers_['categorical'].named_steps['onehot'].get_feature_names_out(input_features=categorical_col)

# Obter os nomes das colunas numéricas
numerical_feature_names = num_col
remainder_col_names = [col for col in df_train.columns if col not in (num_col+categorical_col)]

# Combinar os nomes das colunas categóricas e numéricas
all_feature_names = list(categorical_feature_names) + numerical_feature_names + remainder_col_names
all_feature_names

In [ ]:
df_test_transformed_with_columns = pd.DataFrame(df_test_transformed, columns=all_feature_names)
df_val_transformed_with_columns = pd.DataFrame(df_val_transformed, columns=all_feature_names)
df_train_transformed_with_columns = pd.DataFrame(df_train_transformed, columns=all_feature_names)

In [ ]:
df_train_transformed_with_columns